In [1]:
import tweepy
import pandas as pd
import time

In [ ]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAACq55QEAAAAA0n0%2BnV88MBYetDKWNnW2JBSXiPc%3DljYGemTcO0KLMwfBL9thVAJcviYT6ggW5LpcnRe8JFRYStAlWo"

client = tweepy.Client(BEARER_TOKEN, wait_on_rate_limit=True)

query = '("Pertamax Green" OR "Pertamax Green 95" OR bioetanol OR "bensin etanol" OR "bensin campur tebu") lang:id -is:retweet'

tweet_data_all = []
tweet_count = 0
max_tweet = 100

In [ ]:
for response in tweepy.Paginator(client.search_recent_tweets,
                                 query=query,
                                 tweet_fields=[
                                     "created_at", 
                                     "public_metrics", 
                                     "author_id", 
                                     "lang", 
                                     "referenced_tweets"
                                 ],
                                 expansions=[
                                     "author_id",
                                     "referenced_tweets.id" 
                                 ],
                                 user_fields=[
                                     "username", 
                                     "name", 
                                     "public_metrics" # Untuk followers_count
                                 ],
                                 
                                 max_results=100,
                                 limit=(max_tweet // 100) + 1):

    users_lookup = {}
    if response.includes and 'users' in response.includes:
        users_lookup = {user.id: user for user in response.includes['users']}

    if response.data:
        for tweet in response.data:
            if tweet_count < max_tweet:
                
                # Mengambil data pengguna dari lookup
                author_info = users_lookup.get(tweet.author_id)
                username = author_info.username if author_info else "N/A"
                author_followers = author_info.public_metrics['followers_count'] if (author_info and author_info.public_metrics) else 0
                
                # Mengecek jenis tweet (reply/quote?)
                tweet_type = "Original"
                referenced_tweet_id = None
                if tweet.referenced_tweets:
                    tweet_type = tweet.referenced_tweets[0].type
                    referenced_tweet_id = tweet.referenced_tweets[0].id

                # Menambahkan data ke list
                tweet_data_all.append({
                    'id': tweet.id,
                    'created_at': tweet.created_at,
                    'text': tweet.text,
                    'author_id': tweet.author_id,
                    'username': username,
                    'author_followers': author_followers,
                    'likes': tweet.public_metrics['like_count'],
                    'retweets': tweet.public_metrics['retweet_count'],
                    'replies': tweet.public_metrics['reply_count'],
                    'tweet_type': tweet_type,
                    'referenced_tweet_id': referenced_tweet_id,
                        
                })
                tweet_count += 1
            else:
                break
    else:
        print("Tidak ada data ditemukan pada halaman ini.")

    if tweet_count >= max_tweet:
        print(f"Batas maksimum {max_tweet} tweet tercapai.")
        break
    
    time.sleep(1) 

print(f"Total tweet terkumpul: {len(tweet_data_all)}")

df = pd.DataFrame(tweet_data_all)
    
print("\n--- Contoh Data Terkumpul (dengan data pengguna) ---")
print(df[['created_at', 'text', 'username', 'author_followers', 'likes', 'tweet_type']].head())

nama_file = "tweets_kebijakan_pemerintah_lengkap2.csv"
df.to_csv(nama_file, index=False, encoding='utf-8-sig')

Rate limit exceeded. Sleeping for 877 seconds.
Rate limit exceeded. Sleeping for 899 seconds.


Total tweet terkumpul: 178

--- Contoh Data Terkumpul (dengan data pengguna) ---
                 created_at  \
0 2025-11-14 12:34:03+00:00   
1 2025-11-14 10:23:00+00:00   
2 2025-11-14 09:55:08+00:00   
3 2025-11-14 08:27:47+00:00   
4 2025-11-14 07:25:53+00:00   

                                                text       username  \
0  @NickVesting @SinarCahaya84 @Areanesia24 Ya, B...           grok   
1  Meski biofuel bukan hal baru. Pertamina sendir...       xremiums   
2  Minta tolong info spbu plat merah yang ready p...        sbyfess   
3  @BangWilmar @sharpandshark Ya, bahan baku Bobi...           grok   
4  Molase tetes tebu adalah bahan multifungsi den...  widyapermahfm   

   author_followers  likes  tweet_type  
0           6630174      0  replied_to  
1              2934      3  replied_to  
2            669661     26    Original  
3           6630174      0  replied_to  
4                 0      0    Original  
